# Striped Impulse

Esse é o nome-codigo inicial de um engine criado utilizando o Ursina.
O Ursina é uma camada de abstração em Python que executa em cima do Panda3D e simplifica bastante o trabalho.
O desenvolvimento será em Python e nao será embutido em notebooks, mas documentarei partes relevantes. No entanto, vou utilizar outros notebooks pra isso.
Neste pretendo colocar ferramentas para o engine em si.
Inicialmente, duas: o Editor SVG para desenhar cidades, e um criador de imagens para Sky Spheres.
